In [36]:
import tomli
import pandas as pd
from apsim.op_manager import OpManager, get_date
import xml.etree.ElementTree as ET

In [44]:
with open("mgmt_example.toml", "rb") as f:
        mgmt = tomli.load(f)

In [45]:
all_yearly_mgmt = mgmt['mgmt']['year']

In [46]:
ops = ["plant", "fert", "tillage", "harvest"]

In [70]:
op_man = OpManager()

In [71]:
for year in all_yearly_mgmt:
    #get mgmt for a given year
    year_mgmt = all_yearly_mgmt[year]
    #add planting ops
    for year_id in year_mgmt['plant']:
        planting_mgmt = year_mgmt['plant'][year_id]
        op_man.add_plant_op(
            get_date(planting_mgmt['planting_date'], year),
            planting_mgmt['sow_crop'],
            planting_mgmt['sowing_density'],
            planting_mgmt['sowing_depth'],
            planting_mgmt['cultivar'],
            planting_mgmt['row_spacing'],
            )
    #add fertiliser ops
    for year_id in year_mgmt['fert']:
        fert_mgmt = year_mgmt['fert'][year_id]
        if fert_mgmt['kg_ha'] != None and fert_mgmt['kg_ha'] > 0.0:
            op_man.add_fert_op(
            get_date(fert_mgmt['date'], year),
            fert_mgmt['kg_ha'],
            fert_mgmt['depth'],
            fert_mgmt['formula'],
            )
    #add tillage ops
    for year_id in year_mgmt['tillage']:
        till_mgmt = year_mgmt['tillage'][year_id]
        if till_mgmt['depth'] != None and till_mgmt['residue_incorp'] > 0.0:
            op_man.add_till_op(
                get_date(till_mgmt['date'], year),
                till_mgmt['implement'],
                till_mgmt['residue_incorp'],
                till_mgmt['depth'],
                )
    #add harvest ops
    for year_id in year_mgmt['harvest']:
        harvest_mgmt = year_mgmt['harvest'][year_id]
        op_man.add_harvest_op(
            get_date(harvest_mgmt['date'], year),
            harvest_mgmt['crop']
        )

In [72]:
ET.indent(op_man.man_xml)
# ET.dump(op_man.man_xml)
print(ET.tostring(op_man.man_xml, encoding='unicode'))

<folder name="Manager folder">
  <operations name="Operations Schedule">
    <operation condition="start_of_day">
      <date>16/5/2021</date>
      <action>soybean sow plants = 34 (plants/m2), sowing_depth = 51 (mm), cultivar = MG_2, row_spacing = 762 (mm), crop_class = plant</action>
    </operation>
    <operation condition="start_of_day">
      <date>28/4/2021</date>
      <action>SurfaceOrganicMatter tillage type = user_defined, f_incorp = 0.25 (0-1), tillage_depth = 88 (mm)</action>
    </operation>
    <operation condition="start_of_day">
      <date>16/5/2021</date>
      <action>SurfaceOrganicMatter tillage type = user_defined, f_incorp = 0.33 (0-1), tillage_depth = 76 (mm)</action>
    </operation>
    <operation condition="start_of_day">
      <date>24/9/2021</date>
      <action>soybean end_crop</action>
    </operation>
    <operation condition="start_of_day">
      <date>18/5/2022</date>
      <action>maize sow plants = 8 (plants/m2), sowing_depth = 51 (mm), cultivar = B_